# Custom Class Example

This example uses several other python files in the same folder.

In [1]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp # our custom app

from trulens_eval.tru_custom_app import TruCustomApp

from trulens_eval import Tru
# Tru object manages the database of apps, records, and feedbacks; and the
# dashboard to display these.
tru = Tru()

tru.reset_database()

ModuleNotFoundError: No module named 'examples'

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # Optional alternative to decorators:
    #methods_to_instrument={
    #    ca.respond_to_query: Select.Query(), # paths relative to "app"
    #    ca.retrieve_chunks: Select.Query(),
    #    ca.retriever.retrieve_chunks: Select.Query().retriever
    #},
    ## Add extra data to show up as app serialization. See tru_custom_app.py documentation.
    #app_extra_json=dict(
    #    name="This is my custom app. Anything provided to app_extra_json will be merged into the serialization of app",
    #)
)


In [ ]:
# Show instrumented components and methods.

ta.print_instrumented()

In [ ]:
# Normal usage:
# ca.respond_to_query("What is the capital of Indonesia?")

# Instrumented usage:
response, record = ta.with_record(
    ca.respond_to_query, input="What is the capital of Indonesia?"
)


In [ ]:
# Show the app output:

response

In [ ]:
# Show the instrumentation record.

record.dict()

In [ ]:
# Start the dasshboard. If you running from github repo, you will need to adjust
# the path the dashboard streamlit app starts in by providing the _dev argument.
tru.start_dashboard()